In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
!pip3 install statsmodels

In [ ]:
df = pd.read_csv('HousingData.csv')

In [ ]:
# to view all columns of the database in the output window
pd.set_option('display.max_columns',None)

In [ ]:
# seeing which all features have missing values
df.isna().sum()

In [ ]:
# the columns with missing values are CRIM, ZN, INDUS, CHAS, AGE, and LSTAT, each with 20 missing values. The target variable MEDV does not have any missing values.

# Data Imputation

In [ ]:
df

In [ ]:
df.columns

In [ ]:
# Can we drop rows which have NaN values?
missing_row_count = df.isna().any(axis=1).sum() # Count of rows with at least one missing (NaN) value
print(missing_row_count)
#number of rows in the dataframe
len(df)

#since dropping 112 rows would make our dataset very small, we will impute missing values in the features

In [ ]:
"""
Plotting all features with missing values on the y-axis against row index helps us:

- Visually inspect the **shape and structure** of each feature (e.g., smooth, noisy, or plateaued).
- Identify whether features exhibit **collinearity** or similar patterns — useful when deciding KNN-based imputation.
- Detect features that are **too erratic or zigzagged**, where local imputation (like KNN or interpolation) may be unreliable.
- Understand whether values change **gradually or abruptly** across the dataset, which can influence imputation strategy.

This kind of visualization provides intuition for selecting the most appropriate imputation method: 
mean/median for noisy distributions, KNN for locally structured features, and possibly dropping rows/features that are unstable or uninformative.
"""

"""
Should we normalise before plotting?
No
When you're visually inspecting each feature individually, you usually want to preserve their original scale and units, because that reflects the real-world meaning (e.g., AGE is in %, TAX is per $10k).
You don't want to flatten or distort shapes via scaling before assessing their structure.
For plots like your value vs. index, it's good to see actual values, not rescaled ones.
"""

# Features with missing values
missing_features = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'AGE', 'LSTAT']
colors = cm.get_cmap('tab10', len(missing_features))  # Clean color palette

plt.figure(figsize=(14, 6))

for i, feature in enumerate(missing_features):
    plt.plot(df.index, df[feature], label=feature, color=colors(i), linewidth=1.5)

plt.title("Features with Missing Values", fontsize=16)
plt.xlabel("Row Index", fontsize=12)
plt.ylabel("Value", fontsize=12)
plt.legend(title="Feature", fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
"""
Exploring collinearity between AGE and LSTAT to assess whether one can be used to impute the other.
1. However, since both features contain missing values, using one to impute the other may introduce circular logic.
2. This is problematic because each depends on incomplete information, making such imputation unreliable without prior handling.

What can be done instead:
- Impute one with a simpler method first (e.g., median for AGE),
- Then use it to impute the other (e.g., LSTAT via regression or KNN),
- Or exclude rows where both are missing if that’s acceptable and rare.
"""
df[['AGE', 'LSTAT']].corr()

In [ ]:
# imputing CHAS
df['CHAS'].value_counts()
# since CHAS is a categorical feature which is heavily imbalanced towards 0, we can impute missing values with 0, as it reflects the mode. And avoids introducing rare class bias (imputing 1s by mistake).
chas_mode = df['CHAS'].mode()[0]
df['CHAS'].fillna(chas_mode, inplace=True)

In [ ]:
"""
Zigzag / Noisy patterns

Features like AGE and ZN jump rapidly, without smooth local structure.
→ These are bad candidates for KNN or interpolation, because:

Nearby rows (in index) don't behave similarly.
No smoothness = KNN may pull in totally unrelated neighbors
"""

In [ ]:
# imputing for AGE 
print("Mean AGE:", df['AGE'].mean())
print("Median AGE:", df['AGE'].median())
df['AGE'].hist(bins=20)
#since the histogram is so right-skewed, the mean would be quite skewed, so, we will use median to impute missing age values.
df['AGE'].fillna(df['AGE'].median(), inplace=True)

In [ ]:
# imputing for ZN
print("Mean ZN:", df['ZN'].mean())
print("Median ZN:", df['ZN'].median())
df['ZN'].hist(bins=20)
# most zn values are 0, so we can impute with the median 0. non zero values are sparse and scattered
df['ZN'].fillna(df['ZN'].median(), inplace=True)

In [ ]:
"""imputing LSTAT
While LSTAT (percentage of lower-status population) is less erratic than AGE or ZN, it's still:
Right-skewed (long tail on the high end),
Not smoothly varying enough to trust interpolation,
And lacks strong, consistent correlation with other features (as shown in your scatter plots).
"""
print("Mean LSTAT:", df['LSTAT'].mean())
print("Median LSTAT:", df['LSTAT'].median())
df['LSTAT'].hist(bins=20)
df['LSTAT'].fillna(df['LSTAT'].median(), inplace=True)

In [ ]:
# imputing for INDUS
# first we plot INDUS values against row index to see if any pattern stands out
plt.figure(figsize=(12, 5))
plt.plot(df.index, df['INDUS'], linestyle='-', marker='o', markersize=3, color='steelblue')
plt.title("INDUS Values Across Row Index")
plt.xlabel("Row Index")
plt.ylabel("INDUS (Proportion of non-retail business acres)")
plt.grid(True)
plt.show()

In [ ]:
# ---------------------------------------
# IMPUTATION STRATEGY FOR 'INDUS' FEATURE
# ---------------------------------------

# When we plotted INDUS against row index, we observed long plateaus — e.g., stretches where 
# many rows had the same INDUS value like 18.1 or 19.58, followed by sudden changes. 
# This gave us the intuition that these plateaus might not be random — maybe they’re the result 
# of natural groupings in the data (like similar towns or zones), and those groupings might also 
# be reflected in other features like TAX, NOX, AGE, or DIS.

# That led us to consider using KNN or clustering for imputation:
# If rows with similar INDUS values also tend to have similar values in other features,
# then we could find "neighborhoods" in feature space and use them to impute missing INDUS values.

# However, this is just a visual hypothesis based on the index — which may have no geographic or 
# neighborhood meaning. The plateaus might be real, or INDUS could be largely independent. 
# We cannot assume KNN or clustering will work unless we test this properly.

# To justify KNN:
# - We would first check individual correlations:
#     df.corr()['INDUS'].sort_values(ascending=False)
#   and select features with moderate correlation (|r| > 0.3).
# - Even if no individual feature is highly correlated, combinations of features (e.g., TAX + NOX + AGE)
#   might form meaningful neighborhoods — which KNN could use.
# - To explore this, we would need dimensionality reduction tools like PCA, t-SNE, or UMAP to check 
#   if similar INDUS values cluster in reduced feature space.
# - We could also validate clustering structure using silhouette scores or visual inspection.

# Practical limitations with KNN or clustering:
# - We must exclude the target variable MEDV to prevent data leakage during imputation.
# - We must also exclude CRIM since it has missing values — KNN requires all features to be non-null.
# - Not all features are guaranteed to help predict INDUS — including irrelevant ones may add noise.
# - Scaling is required since KNN is distance-based.
# - Both KNN and clustering require tuning, validation, and extra computation.

# Clustering (e.g., KMeans) is another option — group similar rows and impute INDUS with the group’s mean —
# but it comes with the same overhead: handling missing values, feature selection, preprocessing, and validation.

# Given all this effort — and the fact that INDUS has only 20 missing values — we choose a simple, safe
# approach for now:
#     - Use **mode** (most frequent value) for imputation — since INDUS has dominant repeated values.
#     - Alternatively, use **median** if distribution appears continuous but skewed.

# Later, once the model is trained, we can use SHAP values or other interpretability tools to evaluate 
# how important INDUS actually was. If it turns out to be significant, we can revisit its imputation 
# and apply a more rigorous method like KNN or clustering-based approaches.

# FINAL IMPUTATION (for now):
df['INDUS'].fillna(df['INDUS'].mode()[0], inplace=True)

In [ ]:
# Imputing CRIM
# Overlay missing points in red Xs on plot of CRIM values against row indices.
missing_indices = df[df['CRIM'].isna()].index
plt.figure(figsize=(12, 5))
plt.plot(df.index, df['CRIM'], linestyle='-', marker='o', markersize=3, color='darkred', label='CRIM')
plt.scatter(missing_indices, [0]*len(missing_indices), color='black', marker='x', label='Missing')
plt.title("CRIM Values Across Row Index (Missing Highlighted)")
plt.xlabel("Row Index")
plt.ylabel("CRIM")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
"""
CRIM is mostly very low, clustered near 0, for the majority of rows.
Around row 360–460, there’s a sharp rise with many spikes up to 80–90.
The missing values are scattered throughout, including in both the low-CRIM and high-CRIM zones.
So there are clearly two major regimes:
- Low-crime zones (dense, flat region near 0)
- High-crime zones (spiky cluster later in index)

Possible Approachs:
1.  Clustering-Based Imputation
    Cluster rows into 2–3 groups using all complete features, and impute CRIM using the average (or median) of the cluster the row belongs to.
    Why it makes sense:
    The plot clearly suggests CRIM behaves in clusters.
    If neighborhoods with high TAX/NOX/AGE correspond to high CRIM, clustering will capture that.
2.  Classification-Based Imputation
    Treat imputation as a supervised task: train a model to predict CRIM range (e.g., low vs high crime), and use that to infer missing values.
    Feels overkill, especially for only 20 missing values — and not ideal unless you're imputing categories, not continuous values.
3.  Index-Based Averaging (Local Neighborhoods)
    Use a simple moving window (±2 or ±3 rows) to take the average of nearby non-null values.
    This is quick and may work well for the low-CRIM areas, where values are stable.
    But:
    Around high-crime regions (spiky zone), this may be too unstable
    Still ignores feature similarity, only uses position
    So it's fine as a "quick fix", but not ideal for robust modeling. 
4.  Mode / Median Imputation
    This would ignore the obvious structure and plateau regions — so while it's quick and safe, you’d lose too much information.
"""

In [ ]:
# 1. Select features (exclude CRIM and MEDV)
features_for_clustering = ['ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 
                           'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']  # All complete now

# 2. Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(df[features_for_clustering])

# 3. Fit KMeans
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df['CRIM_cluster'] = kmeans.fit_predict(X)

# Create a list to store comparison info for previously missing CRIM rows
crim_comparison = []

# 4. Impute CRIM and track comparison with window mean
for cluster_label in df['CRIM_cluster'].unique():
    # Get indices of missing CRIM values in this cluster BEFORE filling
    missing_indices = df[
        (df['CRIM_cluster'] == cluster_label) & (df['CRIM'].isna())
    ].index

    # Compute cluster median from CRIM values that are not missing
    cluster_median = df.loc[
        (df['CRIM_cluster'] == cluster_label) & (df['CRIM'].notna()), 'CRIM'
    ].median()

    for idx in missing_indices:
        # Impute using cluster median
        df.at[idx, 'CRIM'] = cluster_median

        # Compute rolling window mean (excluding current row)
        window_range = range(max(0, idx - 2), min(len(df), idx + 3))
        window_vals = df.loc[list(window_range), 'CRIM']
        window_vals = window_vals.drop(idx)

        if not window_vals.empty:
            window_mean = window_vals.mean()
            diff = abs(cluster_median - window_mean)

            crim_comparison.append({
                'Index': idx,
                'CRIM_cluster_imputed': cluster_median,
                'CRIM_window_mean': window_mean,
                'Difference': diff
            })

# 5. Convert comparison results to DataFrame
crim_comparison_df = pd.DataFrame(crim_comparison)

# 6. Visualize clusters in 2D using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['CRIM_cluster'], cmap='Accent', alpha=0.7)
plt.title("KMeans Clusters in 2D PCA Space (excluding CRIM and MEDV)")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.legend(*scatter.legend_elements(), title="Cluster")
plt.grid(True)
plt.show()

In [ ]:
# Adopted cluster-wise median imputation for CRIM based on feature similarity using KMeans clustering.
# As a lightweight secondary validation, we compared the imputed values against a rolling window mean 
# (±2 rows) based on row index. While the window mean is simpler and purely local, it tends to fluctuate 
# more, especially in high-CRIM regions with sharp spikes. The comparison confirmed that cluster median 
# imputations are generally more stable and better aligned with feature-based groupings.
crim_comparison_df

In [ ]:
"""⚠️ Note: The code blocks below for imputing CRIM assume that CRIM is the only column with missing values.
If other features in the dataframe also contain missing values, clustering-based or KNN-based imputation for CRIM will fail,
since these methods rely on other features (which must be complete) to compute similarity or distance.
"""


# Plot CRIM values with wherein CRIM value is the y-axis and row index is the x-axis
"""
plt.figure(figsize=(12, 5))
plt.plot(df.index, df['CRIM'], marker='o', linestyle='-', label='CRIM')
plt.xlabel('Row Index')
plt.ylabel('CRIM Value')
plt.title('CRIM values across dataset')
plt.legend()
plt.grid(True)
plt.show()
"""


"""
Simple imputation (mean, median) or linear interpolation won't work here because CRIM is highly skewed:
Most values are near 0, but there's a sharp spike around index 370–420.
Mean/median would bias estimates too low, and interpolation would miss abrupt regional jumps.
Imputation must account for local context — clustering or KNN is better suited for this distribution.
"""

"""
We intentionally exclude the 'CRIM' column when performing clustering for imputation.

Including 'CRIM' in clustering would lead to two key issues:
1. You can't compute distances for rows where 'CRIM' is missing — so you can't assign those rows to clusters.
2. Even worse, if 'CRIM' is a major factor driving the separation between clusters (e.g., some clusters have high crime rates, others low), 
   using it during clustering introduces bias. You're using the very value you're trying to impute to define the group it belongs to — 
   which is a form of data leakage and circular logic.

Instead, we use only contextual features (like RM, LSTAT, TAX, etc.) to build clusters. This allows us to impute missing 'CRIM' values 
based on the typical behavior of similar neighborhoods, inferred through available variables.

This approach is not perfect, but it's valid — because we're using indirect signals to estimate 'CRIM', rather than CRIM itself.

To further improve reliability, we layer two imputation methods:
1. K-Means Clustering:
   - Groups samples based on contextual patterns.
   - Imputes missing 'CRIM' using cluster-level statistics (e.g., mean or median).
   - Helps understand the global structure — such as which clusters represent high-CRIM vs low-CRIM areas.

2. KNN Imputation:
   - Looks at each sample individually.
   - Uses the most similar neighboring rows (based on other features) to estimate missing 'CRIM'.
   - Acts as a secondary verification or refinement of the cluster-based estimate.

In short, we use clustering to build broader context, and KNN to refine the local estimate. 
Straight KNN alone can sometimes be sensitive to noise and outliers, especially in skewed datasets, so combining both gives more robust imputation.
"""

"""
from sklearn.cluster import KMeans
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# ---------------------------
# Step 1: Define feature set (exclude CRIM)
# ---------------------------
features = ['ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

# Split into complete and missing sets based on CRIM
df_complete = df[df['CRIM'].notna()].copy()
df_missing = df[df['CRIM'].isna()].copy()

# ---------------------------
# Step 2: KMeans Clustering on df_complete
# ---------------------------
scaler = StandardScaler()
X_complete_scaled = scaler.fit_transform(df_complete[features])

# You can tune n_clusters based on silhouette scores
kmeans = KMeans(n_clusters=5, random_state=42)
df_complete['cluster'] = kmeans.fit_predict(X_complete_scaled)

# ---------------------------
# Step 3: Assign clusters to missing rows (based on same features)
# ---------------------------
X_missing_scaled = scaler.transform(df_missing[features])
missing_clusters = kmeans.predict(X_missing_scaled)

# Get average CRIM per cluster
cluster_crim_map = df_complete.groupby('cluster')['CRIM'].mean()

# Impute CRIM in missing rows using cluster averages
df_kmeans_imputed = df.copy()
df_kmeans_imputed.loc[df['CRIM'].isna(), 'CRIM'] = [
    cluster_crim_map[cluster] for cluster in missing_clusters
]

# ---------------------------
# Step 4: KNN Imputation
# ---------------------------
# Scale all features except CRIM
scaler_knn = StandardScaler()
scaled_features = scaler_knn.fit_transform(df[features])

# Combine scaled features + CRIM for KNN
df_knn_input = pd.DataFrame(scaled_features, columns=features)
df_knn_input['CRIM'] = df['CRIM'].values

# Apply KNN Imputer
knn_imputer = KNNImputer(n_neighbors=5)
df_knn_imputed_array = knn_imputer.fit_transform(df_knn_input)

# Extract only CRIM column after imputation
crim_knn_imputed = df_knn_imputed_array[:, df_knn_input.columns.get_loc('CRIM')]

# ---------------------------
# Step 5: Compare KMeans and KNN imputations
# ---------------------------
missing_indices = df[df['CRIM'].isna()].index

comparison_df = pd.DataFrame({
    'KMeans_CRIM': df_kmeans_imputed.loc[missing_indices, 'CRIM'].values,
    'KNN_CRIM': crim_knn_imputed[missing_indices]
})
comparison_df['Diff'] = np.abs(comparison_df['KMeans_CRIM'] - comparison_df['KNN_CRIM'])

# Summary statistics
print("📊 Imputation Comparison Summary:")
print(comparison_df.describe())

# Optional plot
plt.figure(figsize=(10,5))
plt.plot(comparison_df['KMeans_CRIM'].values, label='KMeans CRIM', marker='o')
plt.plot(comparison_df['KNN_CRIM'].values, label='KNN CRIM', marker='x')
plt.title("Comparison of Imputed CRIM values (Missing Rows)")
plt.xlabel("Index within missing rows")
plt.ylabel("Imputed CRIM")
plt.legend()
plt.grid(True)
plt.show()
"""

# Feature Engineering

In [ ]:
# We used features ['ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'] for clustering
# Since KMeans requires all inputs to be numeric floats, we don't have to check dtype of these columns.
# Since, we have not explored the target columns MEDV yet, we can run some checks on it.
df['MEDV'].apply(type).value_counts()

In [ ]:
# List of features (excluding target)
features = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

# Generate and save plots
for feature in features:
    plt.figure(figsize=(6, 4))
    plt.scatter(df[feature], df['MEDV'], alpha=0.5)
    plt.title(f'{feature} vs MEDV')
    plt.xlabel(feature)
    plt.ylabel('MEDV')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    plt.close()  # Close to prevent overlapping plots in notebook

In [ ]:
# -------------------------------------------------------------
# WHY WE PLOT FEATURES AGAINST TARGET USING ORIGINAL VALUES
# -------------------------------------------------------------
# We do *not* plot features after standard scaling. Instead, we use the original,
# unscaled (but imputed) dataset to visualize how each feature relates to the target (MEDV).
#
# Here's why:
# 📊 Interpretability: We want to understand the real-world relationship
#     between features like RM (number of rooms) or TAX (property tax) and MEDV.
# 🔍 Meaningful axes: After scaling, features lose their original units and are centered 
#     around 0 with std ~1. The plot becomes harder to interpret.
# 🚫 Target isn't scaled: Since MEDV remains in original units (e.g., $1000s), plotting 
#     it against scaled features would be visually misleading.
#
# -------------------------------------------------------------
# FEATURE ENGINEERING DECISIONS BASED ON SCATTER PLOTS
# -------------------------------------------------------------
# By plotting each feature vs. MEDV, we identified patterns that suggest non-linear 
# relationships. To help linear regression capture these, we applied transformations:
#     - df['LOG_CRIM']  = np.log1p(df['CRIM'])     # CRIM is skewed → log transform
#     - df['LSTAT_SQ']  = df['LSTAT'] ** 2         # LSTAT has a curved relation → square
#     - df['RM_SQ']     = df['RM'] ** 2            # RM may show non-linearity → square
#
# Other features showed weak or noisy relationships with MEDV, so we kept them as-is.

# -------------------------------------------------------------
# WHY SOME FEATURES (e.g., ZN, CHAS, RAD) ARE NOT TRANSFORMED
# -------------------------------------------------------------
# Some features like 'ZN' (proportion of residential land zoned), 'CHAS' (Charles River dummy),
# and 'RAD' (index of accessibility to radial highways) showed **vertical lines** in their
# scatter plots with MEDV. This means:
#   - These features take on only a **few discrete values**
#   - For each value, MEDV varies across a wide range
#
# ✳️ These features are **categorical or ordinal in nature** — transforming them using
# log or squaring would be meaningless and possibly harmful.
#
# 👉 It's better to **keep them as-is**, since:
#   - They're already encoded meaningfully (e.g., 0/1 for CHAS)
#   - Their interaction with the target might be captured through model coefficients
#   - Attempting to "linearize" them doesn't help, because the feature's informativeness
#     comes from **category differences**, not continuous scale

# -------------------------------------------------------------
# WHEN AND WHY TO DO FEATURE ENGINEERING
# -------------------------------------------------------------
# ➤ Feature engineering is especially important for **linear regression** models, 
#    which can only model straight-line (linear) relationships between features and target.
#
# ➤ By transforming features, we try to make **nonlinear relationships look linear**, 
#    allowing the linear model to perform better.
#
# ➤ In more complex models like **SVMs (with kernels)**, **decision trees**, or **XGBoost**, 
#    this step is often unnecessary — those models can naturally handle non-linear patterns.
#
# ➤ So for linear regression, feature engineering becomes a powerful and necessary tool 
#    to build a robust, predictive model.

# -------------------------------------------------------------
# PCA: WHEN TO USE IT AND WHY IT HELPS
# -------------------------------------------------------------
# PCA (Principal Component Analysis) is a dimensionality reduction technique.
# It can be helpful when:
#     - You have many correlated features (collinearity)
#     - You want to reduce feature count and avoid overfitting
#     - You don't need interpretability for individual features
#
# 🔄 PCA works by combining correlated features into **principal components** —
#     new features that are orthogonal (i.e., uncorrelated with each other).
#     This removes redundancy and helps address **multicollinearity**, which can
#     be a major problem in linear models (e.g., unstable coefficients, high variance).
#
# 📉 Instead of dropping features manually, PCA lets us project the data into
#     a smaller number of components that still explain most of the variance.
#     This often results in simpler models that generalize better.
#
# In our case, we haven’t used PCA yet — but it could be useful later if we
# decide to:
#     - Reduce dimensionality
#     - Denoise the dataset
#     - Prepare the data for clustering, SVMs, or regularized linear models


In [ ]:
df['LOG_CRIM'] = np.log1p(df['CRIM']) 
df['LSTAT_SQ'] = df['LSTAT'] ** 2
df['RM_SQ'] = df['RM'] ** 2

In [ ]:
# Pairs of (original, transformed) features
feature_pairs = [
    ('CRIM', 'LOG_CRIM'),
    ('LSTAT', 'LSTAT_SQ'),
    ('RM', 'RM_SQ')
]

plt.figure(figsize=(15, 9))

for i, (raw_feat, eng_feat) in enumerate(feature_pairs):
    # Left: original feature vs MEDV
    plt.subplot(3, 2, 2*i + 1)
    plt.scatter(df[raw_feat], df['MEDV'], alpha=0.6, s=20, color='gray')
    plt.xlabel(raw_feat)
    plt.ylabel('MEDV')
    plt.title(f'{raw_feat} vs MEDV')
    plt.grid(True)

    # Right: engineered feature vs MEDV
    plt.subplot(3, 2, 2*i + 2)
    plt.scatter(df[eng_feat], df['MEDV'], alpha=0.6, s=20, color='teal')
    plt.xlabel(eng_feat)
    plt.ylabel('MEDV')
    plt.title(f'{eng_feat} vs MEDV')
    plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
"""
Did feature engineering transformations work, ie linearise and smoothen relationship curve between x and target?
1. LSTAT vs MEDV shows a nice downward slope, somewhat linear-ish.
   LSTAT_SQ vs MEDV squashes low LSTAT values together and exaggerates high values, making the relationship more curved and less interpretable.
   Verdict: Don't Keep
2. The original plot (CRIM vs MEDV) is highly skewed and concentrated near zero with some huge outliers (e.g., >80).
   After log1p, the spread is compressed, and the downward trend with MEDV becomes visibly smoother.
   Verdict: Keep
3. RM vs MEDV is already pretty linear and clean.
   RM_SQ vs MEDV stretches values unnecessarily — and makes the plot look more quadratic, not more linear.
   Verdict: Don't Keep
"""

In [ ]:
# Create new transformed features
df['INV_LSTAT'] = 1 / df['LSTAT'] 
df['LOG_LSTAT'] = np.log1p(df['LSTAT'])   
df['LOG_RM'] = np.log(df['RM'])
df['EXP_INV_RM'] = np.exp(-df['RM']) 

In [ ]:
# Plotting
plt.figure(figsize=(18, 8))

# --- Row 1: LSTAT comparisons ---
lstat_feats = ['LSTAT', 'LOG_LSTAT', 'INV_LSTAT']
for i, feature in enumerate(lstat_feats):
    plt.subplot(2, 3, i + 1)
    plt.scatter(df[feature], df['MEDV'], alpha=0.6, s=20, color='purple')
    plt.xlabel(feature)
    plt.ylabel('MEDV')
    plt.title(f'{feature} vs MEDV')
    plt.grid(True)

# --- Row 2: RM comparisons ---
rm_feats = ['RM', 'LOG_RM', 'EXP_INV_RM']
for i, feature in enumerate(rm_feats):
    plt.subplot(2, 3, i + 4)
    plt.scatter(df[feature], df['MEDV'], alpha=0.6, s=20, color='darkgreen')
    plt.xlabel(feature)
    plt.ylabel('MEDV')
    plt.title(f'{feature} vs MEDV')
    plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# these curves again are worse or compress higher values, while only being slightly better. This would lead to loss in interpretability. Hence, we will not feature enigneer columns LSTAT and RM.

In [ ]:
df.columns

In [ ]:
df.drop(['LSTAT_SQ', 'RM_SQ', 'INV_LSTAT', 'LOG_LSTAT', 'LOG_RM', 'EXP_INV_RM'], axis=1, inplace=True)

In [ ]:
df.drop(columns='CRIM', inplace=True)
# Since we will be using LOG_CRIM, we should drop CRIM feature

# Feature Scaling

In [ ]:
# -----------------------------------------------
# SHOULD WE SCALE THE TARGET VARIABLE ('MEDV')?
# -----------------------------------------------

# The short answer: it depends on the type of model you're using.

# WHY WE MIGHT WANT TO SCALE THE TARGET:
# Scaling (e.g., StandardScaler or MinMaxScaler) transforms the target variable 
# to a smaller, more consistent range — which can help models that rely on 
# gradient-based optimization (like neural networks) train more effectively.

# But not all models are sensitive to the scale of the target. Here's the breakdown:

# -------------------------------------------------------
# 1. TREE-BASED MODELS (e.g., RandomForest, XGBoost)
# -------------------------------------------------------
# Do NOT scale the target variable.
# These models split data based on thresholds, not distances or gradients.
# Changing the scale of the target won't affect how the trees split or make predictions.
# Keeping the target in its original units (e.g., thousands of dollars) also helps interpret results.

# -----------------------------------------------------------
# 2. LINEAR MODELS (e.g., Linear Regression, Ridge, Lasso)
# -----------------------------------------------------------
# Scaling the target is OPTIONAL.
# - If your model is solving the normal equation (analytically), scaling isn't needed.
# - If you're using gradient descent (e.g., in PyTorch/TF), scaling might speed up convergence.
# - The downside is: coefficients lose their original units, so interpretability decreases.

# In most practical cases, it's fine to leave the target unscaled, especially 
# when using models like `sklearn.linear_model.LinearRegression`.

# --------------------------------------------------------
# 3. NEURAL NETWORKS / DEEP LEARNING MODELS
# --------------------------------------------------------
# DO scale the target variable.
# These models benefit from both input and output being in a similar scale/range,
# such as [0, 1] or having zero mean and unit variance.
# Without scaling, the model may converge slowly or get stuck due to gradient instability.
# You'll need to inverse-transform the predictions after training to get them 
# back into real-world units.

# -----------------------------------------------------
# TL;DR:
# -----------------------------------------------------
# - Tree models → DON'T scale target
# - Linear models → Optional (depends on solver + convergence)
# - Neural networks → DO scale target (and inverse-transform predictions later)

# For this project (Boston Housing dataset), if you're using linear regression,
# it's fine to keep 'MEDV' in its original units (house price in $1000s) — 
# which is also more interpretable when evaluating predictions.


In [ ]:
# -------------------------------------------------------------
# FEATURE SCALING (StandardScaler)
# -------------------------------------------------------------
# We will apply standard scaling (zero mean, unit variance) to the input features,
# but we will NOT scale the target variable 'MEDV', since we're using linear regression
# and want to keep output predictions in interpretable dollar units ($1000s).

# It's important to perform train-test split BEFORE scaling,
# because scaling depends on the mean and standard deviation of the data.
# If we scale before splitting, we would leak information from the test set
# into the training process, which would give an unrealistic performance estimate.

# Note: We'll also scale the 'CHAS' column, even though it's binary (0/1),
# because StandardScaler will just center it to have zero mean.
# Tree-based models typically don’t require scaling for binary features,
# but for linear models, it's consistent and harmless to scale all features together.

In [ ]:
# Define feature columns and target
features = ['ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'LOG_CRIM']
target = 'MEDV'

# Split dataset into features and target
X = df[features]
y = df[target]

# Split into train and test sets (before any scaling)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply StandardScaler to input features only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Do we need PCA?

In [ ]:
# We only have 13 features in the Boston Housing dataset, and we're using a simple Linear Regression model.
# Unless there's strong multicollinearity (high correlation between features), we don't need PCA.
# PCA is typically used when:
#   - We have a large number of features
#   - Features are highly correlated
#   - We want to reduce dimensionality or noise
#
# In logistic regression, PCA can be useful because orthogonal (uncorrelated) features improve the stability 
# and speed of convergence of the optimization algorithm used to maximize the log-likelihood.
# Orthogonal features also help regularization (L1/L2) work more effectively:
#   - If two features are correlated, a regularized model may struggle to penalize both correctly.
#   - With uncorrelated features, it's easier for the model to shrink irrelevant ones toward zero.

In [ ]:
# ----------------------------------------------
# Understanding Variance Inflation Factor (VIF)
# ----------------------------------------------
# VIF is a tool used to detect multicollinearity in regression problems.
# Multicollinearity happens when two or more features (columns) in your dataset are highly correlated —
# meaning they contain overlapping or redundant information.

# Why is that a problem?
# Imagine you're trying to predict house prices using both "number of rooms" and "total floor area".
# If every house with more rooms also tends to have more floor area, then both features are telling
# the model almost the same thing. The model gets "confused" about how much weight to give each one.

# VIF helps you identify such situations.
# For each feature, VIF measures **how much it can be predicted using the other features**.
# If a feature has a high VIF, it means it is almost a combination of other features —
# and may not be useful on its own.

# Real-life analogy:
# Think of asking two friends for directions — and both give you nearly identical answers.
# Asking both didn't give you new information — in fact, it might just make you second-guess
# or overcomplicate your choice. VIF flags that kind of redundancy.

# Rule of thumb:
# - VIF = 1 → No multicollinearity (ideal)
# - VIF > 5 → Some concern (you may want to look closer)
# - VIF > 10 → Serious multicollinearity (consider dropping or combining features)

# You can think of VIF as a more advanced version of Pearson correlation:
# While Pearson checks the relationship between two variables at a time,
# VIF checks whether a single feature can be predicted from all the others combined.
# It's a more complete measure for identifying redundancy before fitting regression models.

# In this step, we'll compute the VIF for each feature in the scaled training set to check
# whether any are highly correlated and may need to be removed or transformed.


In [ ]:
def compute_vif(X, feature_names):
    vif_dict = {}
    for i in range(X.shape[1]):
        X_i = X[:, i]
        X_other = np.delete(X, i, axis=1)

        # Regress feature i on all others
        model = LinearRegression()
        model.fit(X_other, X_i)
        r_squared = model.score(X_other, X_i)

        vif = 1 / (1 - r_squared) if r_squared < 1 else np.inf
        vif_dict[feature_names[i]] = vif

    return pd.DataFrame({'Feature': list(vif_dict.keys()), 'VIF': list(vif_dict.values())})

# computing VIF on scaled data
vif_df = compute_vif(X_train_scaled, features)
print(vif_df.sort_values(by='VIF', ascending=False))

In [ ]:
# RAD and TAX are known to be highly correlated in the Boston Housing dataset — both relate to infrastructure and zoning.\
"""
Approaches to resolve collinearity:
1. Drop one of RAD or TAX.
2. Keep both but use ridge/lasso regularisation to reduce overfitting, while keeping the features.
3. Apply PCA, if we prefer prefer transforming correlated features into orthogonal ones, but we will lose feature interpretability.
"""

# We start with dropping RAD from the unscaled feature sets
X_train_pruned = X_train.drop(columns=['RAD'])
X_test_pruned = X_test.drop(columns=['RAD'])

# Even though dropping a column like 'RAD' does not change the mean or standard deviation
# of the remaining columns (like 'TAX'), we still need to re-fit the StandardScaler.
# This is because the scaler internally stores the order and number of columns during fit.
# If we drop a column, the positions of other columns shift, and reusing the old scaler
# could apply the wrong scaling to the wrong column.
# So even though the math behind scaling each column stays the same,
# we redo StandardScaler to make sure it lines up correctly with the pruned dataframe.
# REDOING SCALING ON FEATURE SETS

X_train_pruned_scaled = scaler.fit_transform(X_train_pruned)

# Recalculate VIF on re-scaled feature sets to see if it improves
vif_pruned_df = compute_vif(X_train_pruned_scaled, X_train_pruned.columns)
print(vif_pruned_df.sort_values(by='VIF', ascending=False))

In [ ]:
# reduced multicollinearity without using PCA
# collinearity has reduced to <5, which can still be monitored, but is acceptable for traning a model

# Scaling pruned test data using the same scalar
# feature scaling the new input training and testing datasets 
X_test_pruned_scaled = scaler.transform(X_test_pruned)

In [ ]:
X_train_pruned_scaled

In [ ]:
def linear_regression_from_scratch(X, y, learning_rate=0.01, num_epochs = 1000, min_step = 1e-6, verbose = True):
    """
    Implementing Linear Regression from Scratch with detailed equation steps.
    Parameters: 
    X: array of shape(num_samples, num_features) containing training data
    y: array of shape(num_samples, ) containing target values
    learning_rate: float value which is learning rate of gradient descent
    num_epochs: int, maximum number of epochs
    min_step: float, minimum step size for convergence beyond which we stop training
    verbose: bool, whether to print progress

    Returns: 
    weights: array, shape (num_features) Coefficients for the features
    bias: float, intercept term
    loss_history: list, lost history during training
    """
    m, n = X.shape # getting number of samples and features

    # print the initial equation
    if verbose:
        print("Linear Regression Equation:")
        eq = "y = "
        for j in range(n):
            eq += f"w{j+1}*x{j+1} + "
        eq += "b"
        print(eq)
        
        print("\nMean Squared Error (MSE): ")
        print("MSE = (1/m) * Σ(y_i - (w₁*x₁_i + w₂*x₂_i + ... + wₙ*xₙ_i + b))²")
        print("\nGradient for each weight w_j:")
        print("∂MSE/∂w_j = (2/m) * Σ((w₁*x₁_i + w₂*x₂_i + ... + wₙ*xₙ_i + b - y_i) * x_j_i)")
        
        print("\nGradient for bias b:")
        print("∂MSE/∂b = (2/m) * Σ(w₁*x₁_i + w₂*x₂_i + ... + wₙ*xₙ_i + b - y_i)")
        
        print("\nUpdate Rules:")
        print("w_j = w_j - learning_rate * ∂MSE/∂w_j")
        print("b = b - learning_rate * ∂MSE/∂b")
        print("\n" + "-"*50 + "\n")

    #Initialise weights and biases
    # we start with random weights and zero bias
    weights = np.random.randn(n)
    bias = 0.0
    loss_history = []

    # Gradient Descent
    for epoch in range(num_epochs):
        # Step 1: Forward Pass – make predictions using current weight and bias
        # ŷ = w₁x₁ + w₂x₂ + ... + wₙxₙ + b
        y_pred = np.dot(X, weights) + bias

        # Step 2: Calculate the error (difference between predicted and actual target)
        error = y_pred - y  # (ŷ - y)

        # Step 3: Calculate Mean Squared Error
        # MSE = (1/m) * Σ(y_i - ŷ_i)²
        loss = np.mean(error**2)
        loss_history.append(loss)

        # Step 4: Calculate gradients
        # For weights: ∂MSE/∂w_j = (2/m) * Σ((ŷ_i - y_i) * x_j_i)
        gradient_weights = (2/m) * np.dot(X.T, error)

        # For bias: ∂MSE/∂b = (2/m) * Σ(ŷ_i - y_i)
        gradient_bias = (2/m) * np.sum(error)

        # Step 5: Update Weights and Bias
        # w_j = w_j - learning_rate * ∂MSE/∂w_j
        weight_step = learning_rate * gradient_weights
        weights = weights - weight_step

        # b = b - learning_rate * ∂MSE/∂b
        bias_step = learning_rate * gradient_bias
        bias = bias - bias_step

        # checks if ALL weight steps are smaller than our minimum step threshold and bias_step is smaller than minimum step
        # Check for convergence
        if np.all(np.abs(weight_step) < min_step) and np.abs(bias_step) < min_step:
            if verbose:
                print(f"\nConverged at Epoch {epoch}")
            break

        # print progress at every 100 epochs if verbose is set to True
        if verbose and epoch%100 == 0:
            print(f"\nEpoch {epoch}, Loss: {loss:.6f}")
            
            # show the current equation (only at major milestones)
            if epoch%500 == 0:
                eq = f"y = "
                for j in range(n):
                    eq += f"{weights[j]:.4f}*x{j+1} + "
                eq += f"{bias:.4f}"
                print(f"\nCurrent Equation: {eq}")

    # Final Equation:
    if verbose:
        print("\nFinal Linear Regression Equation:")
        eq = f"y = "
        for j in range(n):
            eq += f"{weights[j]:.4f}*x{j+1} + "
        eq += f"{bias:.4f}"
        print(eq)
    
    return weights, bias, loss_history
                    
# --------------------------------------------------------------
# Why do we need a forward pass in gradient descent?
# --------------------------------------------------------------
# The short answer: because of the **chain rule**.
# 
# We're trying to minimize the Mean Squared Error (MSE):
#     MSE = (1/n) * Σ (y_i - ŷ_i)²
# where:
#     ŷ_i = w · x_i + b   ← prediction for each sample i
#
# To update weights using gradient descent, we need:
#     ∂MSE / ∂w_j
# 
# But MSE is a function of predictions (ŷ), not directly of weights.
# So we must apply the **chain rule**:
# 
#     ∂MSE / ∂w_j = ∂MSE / ∂ŷ_i × ∂ŷ_i / ∂w_j
# 
# Let's break that down:
#
# 1. Derivative of MSE w.r.t. prediction:
#       ∂MSE / ∂ŷ_i = 2 * (ŷ_i - y_i)
# 
# 2. Derivative of prediction w.r.t. weight w_j:
#       ∂ŷ_i / ∂w_j = x_ij  ← the j-th feature of sample i
#
# So to compute the full gradient:
#     ∂MSE / ∂w_j = (2/n) * Σ [ (ŷ_i - y_i) * x_ij ]
#
# Therefore, we **must compute the predictions (ŷ_i)** before we can:
#     - Compute the error (ŷ_i - y_i)
#     - Use the chain rule to get gradients
#
# Conclusion:
# The forward pass is required not to "monitor progress", 
# but because it's **mathematically required** by the **chain rule** 
# to compute how MSE changes with respect to weights.
                

        

In [ ]:
def predict(X, weights, bias):
    # Make predictions using Learned weight and bias
    return np.dot(X, weights) + bias

def evaluate_model(X_test, y_test, weights, bias):
    # Calculate various metrics to evaluate the model on test data
    # Make predictions on test data
    y_pred = predict(X_test, weights, bias)

    # Mean Squared Error
    mse = np.mean((y_test - y_pred)**2)

    # Root Mean Squared Error
    rmse = np.sqrt(mse)

    # Mean Absolute Error
    mae = np.mean(np.abs(y_test - y_pred))
    
    # R² Score (coefficient of determination)
    ss_total = np.sum((y_test - np.mean(y_test))**2)
    ss_residual = np.sum((y_test - y_pred)**2)
    r2 = 1 - (ss_residual / ss_total)

    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    }

In [ ]:
def plot_actual_vs_predicted(y_test, y_pred):
    """
    Plot actual vs predicted values for test data
    
    Parameters:
    -----------
    y_test : array-like
        True target values for test data
    y_pred : array-like
        Predicted values for test data
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    
    # Plot the perfect prediction line
    min_val = min(np.min(y_test), np.min(y_pred))
    max_val = max(np.max(y_test), np.max(y_pred))
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')
    
    plt.title('Test Data: Actual vs Predicted Values')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.grid(True)
    plt.show()

In [ ]:
def plot_loss_history(loss_history):
    """Plot the loss history during training"""
    plt.figure(figsize=(10, 6))
    plt.plot(loss_history)
    plt.title('Loss History (MSE)')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.grid(True)
    plt.show()

In [ ]:
# our data is X_train_pruned_scaled, X_test_pruned_scaled, y_train, y_test

# training the model
weights, bias, loss_history = linear_regression_from_scratch(X_train_pruned_scaled, y_train, learning_rate=0.01, 
                                                             num_epochs=1000, min_step=1e-6, verbose = True)

# Evaluating the model
metrics = evaluate_model(X_test_pruned_scaled, y_test, weights, bias)
print("\nModel Evaluation Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Plot the loss history
plot_loss_history(loss_history)

In [ ]:
# Make predictions on test data
y_pred = predict(X_test_pruned_scaled, weights, bias)

# Plot actual vs predicted values on test data
plot_actual_vs_predicted(y_test, y_pred)

In [ ]:
"""
Learning Rate Effects:

- Linear Models: Affects convergence speed but not final model (if converged)
  • Loss surface is convex (bowl-shaped) with a single global minimum
  • Too large: May oscillate or diverge
  • Too small: Converges very slowly
  • Typical values: 0.001-0.1 (start with 0.01)

- Non-Linear Models: Affects both convergence AND final model quality
  • Different rates can lead to different local minima

- Feature Scaling: Critical for consistent gradient steps
  • Unscaled features → large steps in some dimensions → potential overshooting
  • Always scale features before applying gradient descent
"""

In [ ]:
# testing with a learning rate of 0.1

# training the model
weights, bias, loss_history = linear_regression_from_scratch(X_train_pruned_scaled, y_train, learning_rate=0.1, 
                                                             num_epochs=1000, min_step=1e-6, verbose = True)

# Evaluating the model
metrics = evaluate_model(X_test_pruned_scaled, y_test, weights, bias)
print("\nModel Evaluation Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Plot the loss history
plot_loss_history(loss_history)

# Make predictions on test data
y_pred = predict(X_test_pruned_scaled, weights, bias)

# Plot actual vs predicted values on test data
plot_actual_vs_predicted(y_test, y_pred)

In [ ]:
# very minimal difference, overall model is kinda the same